In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [0]:
from tqdm import tqdm
import os                  
from random import shuffle 
import zipfile
import shutil

In [0]:
#local_zip = '/content/gdrive/My Drive/test1.zip'

#zip_ref = zipfile.ZipFile(local_zip, 'r')

#zip_ref.extractall('/content/gdrive/My Drive/CD2')

#zip_ref.close()

In [0]:
#local_zip2 = '/content/gdrive/My Drive/train.zip'

#zip_ref2 = zipfile.ZipFile(local_zip2, 'r')

#zip_ref2.extractall('/content/gdrive/My Drive/CD2')
#zip_ref2.close()

In [0]:
base_dir='/content/gdrive/My Drive/CD'
train_dir=os.path.join(base_dir,'train')
validation_dir=os.path.join(base_dir,'test1')

train_cats_dir=os.path.join(train_dir,'cats')
train_dogs_dir=os.path.join(train_dir,'dogs')

validation_cats_dir=os.path.join(validation_dir,'cats')
validation_dogs_dir=os.path.join(validation_dir,'dogs')
print(train_cats_dir)

/content/gdrive/My Drive/CD/train/cats


In [0]:
train_cat_fnames = os.listdir(train_cats_dir)
train_dog_fnames = os.listdir(train_dogs_dir)

print(train_cat_fnames[:10])
print(train_dog_fnames[:10])

['cat.107.jpg', 'cat.108.jpg', 'cat.109.jpg', 'cat.110.jpg', 'cat.11.jpg', 'cat.111.jpg', 'cat.112.jpg', 'cat.113.jpg', 'cat.114.jpg', 'cat.115.jpg']
['dog.10.jpg', 'dog.1.jpg', 'dog.100.jpg', 'dog.0.jpg', 'dog.101.jpg', 'dog.102.jpg', 'dog.103.jpg', 'dog.104.jpg', 'dog.105.jpg', 'dog.106.jpg']


In [0]:
def label_img(img):
    word_label = img.split('.')[-3]
    
    if word_label == 'cat': 
        shutil.move(os.path.join(train_dir,img),train_cats_dir)
                                 
    elif word_label == 'dog': 
        shutil.move(os.path.join(train_dir,img),train_dogs_dir)

In [0]:
def create_train_data():
    for img_l in tqdm(os.listdir(train_dir)):
        if(img_l=='dogs' or img_l=='cats'):
            continue
        label = label_img(img_l)

In [0]:
#create_train_data()

In [0]:
print('total training cat images :', len(os.listdir(train_cats_dir)))
print('total training dog images :', len(os.listdir(train_dogs_dir)))
print('total validation cat images :', len(os.listdir(validation_cats_dir)))
print('total validation dog images :', len(os.listdir(validation_dogs_dir)))

total training cat images : 1010
total training dog images : 1000
total validation cat images : 500
total validation dog images : 500


In [0]:
%matplotlib inline

import matplotlib.image as mpimg
import matplotlib.pyplot as plt


In [0]:
import tensorflow as tf

In [0]:
model=tf.keras.models.Sequential([
       tf.keras.layers.Conv2D(16,(3,3),activation='relu',input_shape=(150,150,3)),
       tf.keras.layers.MaxPooling2D(2,2),
       tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
       tf.keras.layers.MaxPooling2D(2,2),
       tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
       tf.keras.layers.MaxPooling2D(2,2),    
       tf.keras.layers.Flatten(),
       tf.keras.layers.Dense(512,activation='relu'),
       tf.keras.layers.Dense(1,activation='sigmoid')
])

In [0]:
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 18496)            

In [0]:
from tensorflow.keras.optimizers import RMSprop
model.compile(optimizer=RMSprop(lr=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen=ImageDataGenerator(rescale=1./255,
                                  rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  fill_mode='nearest')

test_datagen=ImageDataGenerator(rescale=1.0/255.)

train_generator=train_datagen.flow_from_directory(train_dir,
                                                  batch_size=20,
                                                  class_mode='binary',
                                                  target_size=(150,150))
validation_generator=test_datagen.flow_from_directory(validation_dir,
                                                      batch_size=20,
                                                      class_mode='binary',
                                                      target_size=(150,150))


Found 2010 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [0]:
history=model.fit(train_generator,validation_data=validation_generator,
                  steps_per_epoch=100,
                  epochs=2,
                  validation_steps=50,
                  verbose=1
                  )
model.save('/content/gdrive/My Drive/DOGCATtrained1.h5')

Epoch 1/2
100/100 [==============================] - 133s 1s/step - loss: 0.7686 - accuracy: 0.5176 - val_loss: 0.6711 - val_accuracy: 0.6020
Epoch 2/2
100/100 [==============================] - 25s 246ms/step - loss: 0.6758 - accuracy: 0.5784 - val_loss: 0.6327 - val_accuracy: 0.6440


In [0]:
import tensorflow as tf 
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform

Using TensorFlow backend.


In [0]:
model.save('/content/gdrive/My Drive/DOGCATtrained.h5')


In [0]:
model2=tf.keras.models.load_model('/content/gdrive/My Drive/DOGCATtrained_aug.h5')
eval=model2.evaluate(validation_generator)
print("Loss is : ",eval[0])
print("Accuracy is : ",eval[1])

50/50 [==============================] - 4s 90ms/step - loss: 0.4852 - accuracy: 0.7630
Loss is :  0.4852481782436371
Accuracy is :  0.7630000114440918


In [0]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded=files.upload()

for fn in uploaded.keys(): 
  path='/content/' + fn
  img=image.load_img(path, target_size=(150,150))
  image_eg=image.load_img(path)
  x=image.img_to_array(img)
  x=np.expand_dims(x, axis=0)
  images = np.vstack([x])
  
  
  classes = model2.predict(images, batch_size=10)
  plt.imshow(image_eg)
  plt.show()
  print(classes[0])
  
  if classes[0]>0:
    print(fn + " is a doggy")
    
  else:
    print(fn + " is a kitty")

MessageError: ignored

In [0]:
#this is just for checking
for names in train_dog_fnames[:100]:
  path='/content/gdrive/My Drive/CD/train/dogs/' + names
  img=image.load_img(path, target_size=(150, 150))
  x=image.img_to_array(img)
  x=np.expand_dims(x, axis=0)
  images = np.vstack([x])
  
  
  classes = model2.predict(images, batch_size=10)
  print(classes[0])